# Creating cluster member lists

Given a cluster ID, save its member list (i.e., the coordinates of each galaxy in the cluster) as a csv file.

In [1]:
import numpy as np

# path to cluster catalog
cluster_cat_path = "camira_s19a_wide_sm_v1.txt"
# path to cluster members catalog
full_member_cat_path = "camira_mem_s19a_wide_sm_v1.dat"

In [2]:
HSC_cl_cat = np.loadtxt(cluster_cat_path, usecols=(0,1,2,3,4,5)) 
HSC_cl_cat_IDs = np.loadtxt(cluster_cat_path, dtype='str', usecols=(6)) 

full_member_cat = np.loadtxt(full_member_cat_path)

## Dictionary of cluster properties to cluster ID

Key: a 3-value tuple containing the cluster's RA, declination, and redshift (rounded to 3 decimal places) 

Value: the cluster's ID as a string (eg 'C19a1sW02869')

In [3]:
list_props_ID = []

for i in range(len(HSC_cl_cat)):
    RA = HSC_cl_cat[i, 0]
    dec = HSC_cl_cat[i, 1]
    # z in the full member list is rounded to 3 decimal places
    z = np.round(HSC_cl_cat[i, 2], 3)
    ID = HSC_cl_cat_IDs[i]
    props = (RA, dec, z)
    
    list_props_ID.append((props, ID))
    
dict_props_ID = dict(list_props_ID)

## Dictionary of cluster ID to slice in full member list

Key: the cluster's ID as a string (eg 'C19a1sW02869')

Value: slice(a,b); a is the first place in the full member list where the cluster RA appears, 
and b-1 is the last

This dictionary is created as follows: We look for every slice in the full member list with the same (RA, dec, z) tuple. Then, we get the ID via `dict_props_ID`, and we pair the ID with the slice.

In [4]:
## creation of dict_ID_slice
list_ID_slice = []
low_ind = 0 # a
high_ind = 0 # b

num_of_cls = 0

for i in range(len(full_member_cat)):
    cluster_RA = full_member_cat[i, 0]
    cluster_dec = full_member_cat[i, 1]
    cluster_z = full_member_cat[i, 3]
    props = (cluster_RA, cluster_dec, cluster_z)
    
    if (i < (len(full_member_cat) - 1)): 
        if (cluster_RA == full_member_cat[i+1, 0]) and (cluster_z == full_member_cat[i+1, 3]): 
            continue

    # if we've reached this point, it means we've reached the last galaxy 
    # in this cluster
    try:
        ID = dict_props_ID[props]
    except: # account for rounding variation in z
        try:
            props = (cluster_RA, cluster_dec, np.round(cluster_z+0.001,3))
            ID = dict_props_ID[props]
        except:
            props = (cluster_RA, cluster_dec, np.round(cluster_z-0.001,3))
            ID = dict_props_ID[props]
    
    high_ind = i + 1
    list_ID_slice.append((ID, slice(low_ind, high_ind)))
    low_ind = high_ind
    
dict_ID_slice = dict(list_ID_slice)

In [5]:
## check we have the right number of entries in each dict
print(f'number of clusters: {len(dict_props_ID)}')
print(f'size of dict: {len(dict_ID_slice)}')

number of clusters: 16589
size of dict: 16589


In [6]:
def save_mem_list(ID):
    '''
    
    Given a cluster ID, save its member list as a csv.
    
    '''
    prob_min = 0.1
    prob_max = 1
    path = 'Miscentered_Clusters_blank/' + ID + '/' + \
            ID + '_P=' + str(prob_min) + 'to' + str(prob_max) + '.csv'
    member_slice = dict_ID_slice[ID]
    mem_list_no_cut = full_member_cat[member_slice]

    in_prob_range = (mem_list_no_cut[:,7] > prob_min) & (mem_list_no_cut[:,7] <= prob_max)

    mem_list = mem_list_no_cut[in_prob_range][:, 4:6] # columns 4,5 have the gal coords

    np.savetxt(fname=path, X=mem_list, delimiter=',', header='ra, dec')
    
#     print('ID: ' + ID)
#     print(f'Number of galaxies in mem_list: {len(mem_list)}')

In [7]:
# list of miscentered HSC clusters
misc_IDs = ['C19a1sW00048','C19a1sW00191','C19a1sW00577','C19a1sW00825','C19a1sW00866','C19a1sW00955','C19a1sW01237','C19a1sW01272','C19a1sW01400','C19a1sW01596','C19a1sW01711','C19a1sW02098','C19a1sW02209','C19a1sW02334','C19a1sW02498','C19a1sW02659','C19a1sW02852','C19a1sW02863','C19a1sW03332','C19a1sW03508','C19a1sW04184','C19a1sW04937','C19a1sW05441','C19a1sW05554','C19a1sW06808','C19a1sW06984','C19a1sW07320','C19a1sW07325','C19a1sW08037','C19a1sW08159','C19a1sW08325','C19a1sW08377','C19a1sW09451','C19a1sW10081','C19a1sW10241','C19a1sW10675','C19a1sW10803','C19a1sW10814','C19a1sW11430','C19a1sW11629','C19a1sW13410','C19a1sW15645','C19a1sW15696','C19a1sW15945','C19a1sW15996','C19a1sW16502']

In [9]:
for ID in misc_IDs:
    save_mem_list(ID)